## Import libraries:

In [2]:
import numpy as np
import pandas as pd
import statsmodels
import statistics
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.tsa.stattools as ts
import statsmodels.tsa.seasonal as sn
import seaborn as sns; sns.set(style="whitegrid")

from plotly.subplots import make_subplots
import plotly.graph_objects as go

## Load the Data and Preprocess

In [3]:
data =  pd.read_csv("/home/fx/Desktop/Generate_signals/Data Deliverable S&P 500 - Trial 1/S&P 500/SPY with volume.csv")

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7708 entries, 0 to 7707
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       7708 non-null   object 
 1   Open       7708 non-null   float64
 2   High       7708 non-null   float64
 3   Low        7708 non-null   float64
 4   Close      7708 non-null   float64
 5   Adj Close  7708 non-null   float64
 6   Volume     7708 non-null   int64  
dtypes: float64(5), int64(1), object(1)
memory usage: 421.7+ KB


In [5]:
data.describe()

,Open,High,Low,Close,Adj Close,Volume
count,7708.000000,7708.000000,7708.000000,7708.000000,7708.000000,7.708000e+03
mean,167.684173,168.695881,166.570933,167.691657,140.078168,8.450032e+07
std,103.540119,104.107473,102.935280,103.564632,109.663071,9.277685e+07
min,43.343750,43.531250,42.812500,43.406250,24.726740,5.200000e+03
25%,104.777499,105.628748,103.900004,104.904688,70.839341,9.759250e+06
50%,131.665001,132.455002,130.698754,131.601876,93.726135,6.262330e+07
75%,208.942501,209.822506,208.019997,208.919998,181.351856,1.168353e+08
max,479.220001,479.980011,476.059998,477.709991,466.563354,8.710263e+08


In [6]:
# make the Date column as index and drop the Date column
data.index = data["Date"]
data=data.drop(['Date'], axis=1)

In [7]:
# select the stock here
stock = data.copy()# which we want to trade on

#drop null values
stock=stock.dropna()

# Force lowercase (optional)
stock.columns = [x.lower() for x in stock.columns]

In [8]:
target_col = 'close' # set the target column as closing price of stock

In [9]:
stock

,open,high,low,close,adj close,volume
Date,,,,,,
1993-01-29,43.968750,43.968750,43.750000,43.937500,25.029375,1003200
1993-02-01,43.968750,44.250000,43.968750,44.250000,25.207382,480500
1993-02-02,44.218750,44.375000,44.125000,44.343750,25.260780,201300
1993-02-03,44.406250,44.843750,44.375000,44.812500,25.527824,529400
1993-02-04,44.968750,45.093750,44.468750,45.000000,25.634628,531500
...,...,...,...,...,...,...
2023-09-01,453.170013,453.670013,449.679993,451.190002,451.190002,58875700
2023-09-05,450.730011,451.059998,449.170013,449.239990,449.239990,55166200
2023-09-06,448.399994,448.510010,443.809998,446.220001,446.220001,70758500


In this notebook, we will only use two indicators:

## MACD


*   Moving Average Convergence Divergence (MACD) is a trend-following momentum indicator that shows the relationship between two moving averages of a security’s price. 

*   Traders use the MACD to identify when bullish or bearish momentum is high to identify entry and exit points for trades.

*   Comparison of the moving average is done based on three main observations viz convergence, divergence and dramatic rise.



### Calculation of MACD

*   The MACD is calculated by subtracting the 26-period exponential moving average (EMA) from the 12-period EMA.

*   The result of that calculation is the MACD line. A nine-day EMA of the MACD (called the "signal line") is then plotted on top of the MACD line, which can function as a trigger for buy and sell signals. Traders may buy the asset when the MACD crosses above its signal line and sell the asset when the MACD crosses below the signal line.



In [10]:
# Write a Function to calculate the MACD

def get_MACD(df, column, short_ema, long_ema, signal_ema):
    """Return a new DataFrame with the MACD and related information (signal line and histogram)."""
    df['short_ema'] = df[column].ewm(span=short_ema, adjust=False).mean()
    df['long_ema'] = df[column].ewm(span=long_ema, adjust=False).mean()

    # MACD Indicator = 12-Period EMA − 26-Period EMA.
    df['macd'] = df['short_ema'] - df['long_ema']

    # Signal line = 9-day EMA of the MACD line.
    df['signal'] = df['macd'].ewm(span=signal_ema, adjust=False).mean()

    # Histogram = MACD - Indicator.
    df['histogram'] = df['macd'] - df['signal']

    return df


## RSI

*   The RSI provides technical traders with signals about bullish and bearish price momentum, and it is often plotted beneath the graph of an asset’s price.

*   measures the magnitude of recent price changes to evaluate overbought or oversold conditions in the price of an asset. The RSI is displayed as an oscillator (a line graph that moves between two extreme values) and can have a value from 0 to 100.

*   Traditional interpretation and usage of the RSI are that values of 70 or above indicate that a security is becoming overbought or overvalued and may be primed for a trend reversal or corrective pullback in price. An RSI reading of 30 or below indicates an oversold or undervalued condition.



*   Reference:- [link](https://www.investopedia.com/terms/r/rsi.asp)


In [11]:
def get_RSI(df, column, time_window):
    """Return the RSI indicator for the specified time window."""
    diff = df[column].diff(1)

    # This preservers dimensions off diff values.
    up_chg = 0 * diff
    down_chg = 0 * diff

    # Up change is equal to the positive difference, otherwise equal to zero.
    up_chg[diff > 0] = diff[diff > 0]

    # Down change is equal to negative deifference, otherwise equal to zero.
    down_chg[diff < 0] = diff[diff < 0]

    # We set com = time_window-1 so we get decay alpha=1/time_window.
    up_chg_avg = up_chg.ewm(com=time_window - 1,
                            min_periods=time_window).mean()
    down_chg_avg = down_chg.ewm(com=time_window - 1,
                                min_periods=time_window).mean()

    RS = abs(up_chg_avg / down_chg_avg)
    df['rsi'] = 100 - 100 / (1 + RS)

    return df


## Backtest of MACD with RSI:

In [37]:
# We are defining a function which can trade on historical data and calculate the profit/loss at the end
def trade_backtest(stock, short_ema, long_ema, signal_ema, time_window, target_col):
    # If window length is 0, algorithm doesn't make sense, so exit
    if (short_ema == 0) or (long_ema == 0) or (signal_ema == 0):
        return 0

    # Force lowercase (optional)
    stock.columns = [x.lower() for x in stock.columns]

    get_MACD(stock, column=target_col, short_ema=short_ema, long_ema=long_ema, signal_ema=signal_ema)
    get_RSI(stock, column=target_col, time_window=time_window)
    stock_price = stock[target_col]
    stock = stock.dropna()
    print(stock)

    
    count = 0
    print("last price of stock",stock_price[-1])
    money = stock_price[-1] * 1.5
    initial_capital = money
    for i in range(len(stock)):
        """Return the Buy/Sell signal on the specified (price) column (Default = 'close')."""
        if stock['macd'].iloc[i] > stock['signal'].iloc[i] and stock['rsi'].iloc[i] < 70 and money > stock[target_col].iloc[i]:
            count += 1
            money -= 1 * stock_price[i]

        elif stock['macd'].iloc[i] < stock['signal'].iloc[i] and stock['rsi'].iloc[i] > 30 and count > 1: 
            count -= 1
            money += 1 * stock_price[i]
            
        
    money += count*stock_price[-1]
    count = 0
    

    print("Current money: ", money)
    print("Initial capital money: ", initial_capital)

    profit = lambda money, initial_capital : (((money - initial_capital) / initial_capital) * 100)      # Calculate total profit and loss
    print("Totle profit and loss in percentage: ", str(profit(money, initial_capital)) + "%")

    return money, count


money, count = trade_backtest(stock=stock, short_ema=12, long_ema=26, signal_ema=9, time_window=14, target_col='close')

                  open        high         low       close   adj close  \
Date                                                                     
1993-03-12   45.187500   45.218750   44.812500   45.093750   25.688042   
1993-03-15   45.062500   45.312500   45.062500   45.312500   25.812651   
1993-03-16   45.312500   45.437500   45.312500   45.312500   25.812651   
1993-03-17   45.250000   45.250000   44.968750   45.031250   25.652431   
1993-03-18   45.218750   45.500000   45.218750   45.312500   25.812651   
...                ...         ...         ...         ...         ...   
2023-09-01  453.170013  453.670013  449.679993  451.190002  451.190002   
2023-09-05  450.730011  451.059998  449.170013  449.239990  449.239990   
2023-09-06  448.399994  448.510010  443.809998  446.220001  446.220001   
2023-09-07  443.109985  445.549988  442.750000  444.850006  444.850006   
2023-09-08  444.899994  447.109985  444.529999  445.519989  445.519989   

              volume   short_ema    l

## Average Directional Index


*   ADX stands for Average Directional Movement Index and can be used to help measure the overall strength of a trend. The ADX indicator is an average of expanding price range values. The ADX attempts to measure the strength of price movement in positive and negative direction using the DI+ and DI- indicators along with the ADX.


*   Using ADX, we can determine when the trend momentum is strong.  Higher ADX peaks indicate the price momentum is robust, and when the peaks are low, it suggests that the momentum is weakening.

*   Reference:- [link](https://school.stockcharts.com/doku.php?id=technical_indicators:average_directional_index_adx)







In [23]:
def get_adx(stock, lookback):
    high, low, close = stock['high'], stock['low'], stock['close']
    plus_dm = high.diff()
    minus_dm = low.diff()

    plus_dm[plus_dm < 0] = 0
    minus_dm[minus_dm > 0] = 0

    tr1 = pd.DataFrame(high - low)
    tr2 = pd.DataFrame(abs(high - close.shift(1)))
    tr3 = pd.DataFrame(abs(low - close.shift(1)))

    frames = [tr1, tr2, tr3]
    tr = pd.concat(frames, axis = 1, join = 'inner').max(axis = 1)
    atr = tr.rolling(lookback).mean()

    plus_di = 100 * (plus_dm.ewm(alpha = 1/lookback).mean() / atr)
    minus_di = abs(100 * (minus_dm.ewm(alpha = 1/lookback).mean() / atr))
    
    dx = (abs(plus_di - minus_di) / abs(plus_di + minus_di)) * 100
    adx = ((dx.shift(1) * (lookback - 1)) + dx) / lookback
    adx_smooth = adx.ewm(alpha = 1/lookback).mean()

    return plus_di, minus_di, adx_smooth


## Backtest MACD with ADX


1.   A signal to buy will be triggered when the MACD rises above the signal, line with the ADX rising above 25 and the plus_di line crossing above minus_di line.

2.   Similarly, a signal to sell will be triggered when the MACD falls below the signal line, with the ADX rising above 25 and the minus_di line crossing above the plus_di line.



In [26]:
stock =  data.copy()

In [38]:
# We are defining a function which can trade on historical data and calculate the profit/loss at the end
def trade_backtest(stock, short_ema, long_ema, signal_ema, time_window, target_col):
    # If window length is 0, algorithm doesn't make sense, so exit
    if (short_ema == 0) or (long_ema == 0) or (signal_ema == 0):
        return 0

    # Force lowercase (optional)
    stock.columns = [x.lower() for x in stock.columns]

    get_MACD(stock, column=target_col, short_ema=short_ema, long_ema=long_ema, signal_ema=signal_ema)
    stock['plus_di'], stock['minus_di'], stock['adx_smooth'] = get_adx(stock, 14)
    stock_price = stock[target_col]
    stock = stock.dropna()
    print(stock)

    count = 0
    money = 100000
    initial_capital = money
    for i in range(len(stock)):
        """Return the Buy/Sell signal on the specified (price) column (Default = 'close')."""
        # MACD with ADX indicator
        if stock['macd'].iloc[i] > stock['signal'].iloc[i] and money > stock[target_col].iloc[i] and stock['adx_smooth'].iloc[i] > 25 and stock['plus_di'].iloc[i] > stock['minus_di'].iloc[i]:
            count += 1
            money -= 1 * stock_price[i]

        # MACD with ADX indicator
        elif stock['macd'].iloc[i] < stock['signal'].iloc[i] and count > 1 and stock['adx_smooth'].iloc[i] > 25 and stock['minus_di'].iloc[i] > stock['plus_di'].iloc[i]: 
            count -= 1
            money += 1 * stock_price[i]
            
        
    money += count*stock_price[-1]
    count = 0
    

    print("Current money: ", money)
    print("Initial capital money: ", initial_capital)

    profit = lambda money, initial_capital : (((money - initial_capital) / initial_capital) * 100)      # Calculate total profit and loss
    print("Totle profit and loss in percentage: ", str(profit(money, initial_capital)) + "%") 


    return money, count


money, count = trade_backtest(stock=stock, short_ema=12, long_ema=26, signal_ema=9, time_window=14, target_col='close')


                  open        high         low       close   adj close  \
Date                                                                     
1993-03-12   45.187500   45.218750   44.812500   45.093750   25.688042   
1993-03-15   45.062500   45.312500   45.062500   45.312500   25.812651   
1993-03-16   45.312500   45.437500   45.312500   45.312500   25.812651   
1993-03-17   45.250000   45.250000   44.968750   45.031250   25.652431   
1993-03-18   45.218750   45.500000   45.218750   45.312500   25.812651   
...                ...         ...         ...         ...         ...   
2023-09-01  453.170013  453.670013  449.679993  451.190002  451.190002   
2023-09-05  450.730011  451.059998  449.170013  449.239990  449.239990   
2023-09-06  448.399994  448.510010  443.809998  446.220001  446.220001   
2023-09-07  443.109985  445.549988  442.750000  444.850006  444.850006   
2023-09-08  444.899994  447.109985  444.529999  445.519989  445.519989   

              volume   short_ema    l

In [34]:
def plot_MACD(fig, df, row, column=1):
    """Return a graph object figure containing the MACD indicator, the signal line, and a histogram in the specified row."""
    df['Hist-Color'] = np.where(df['histogram'] < 0, 'red', 'green')
    fig.add_trace(go.Bar(x=df.index,
                         y=df['histogram'],
                         name='Histogram',
                         marker_color=df['Hist-Color'],
                         showlegend=True),
                  row=row,
                  col=column)

    fig.add_trace(go.Scatter(x=df.index,
                             y=df['macd'],
                             name='MACD',
                             line=dict(color='darkorange', width=2)),
                  row=row,
                  col=column)

    fig.add_trace(go.Scatter(x=df.index,
                             y=df['signal'],
                             name='Signal',
                             line=dict(color='cyan', width=2)),
                  row=row,
                  col=column)

    fig.update_yaxes(title_text='MACD', row=row, col=column)

    return fig

plot_MACD(fig = make_subplots(rows=2, cols=1), df=stock, row=2, column=1)

## Improve MACD:

> Use MACD Histogram

1.   This is nothing but a difference between the MACD and MACD signal line.
2.   Used to identify the trend(Bullish, bearish) strength


> Combine other technical indicators:-

1.   Moving average
2.   Bollinger Band
3.   Fibonacci retracement


> We can also use other technical indicator rather than RSI:

1.   Stochastic oscillator
2.   Williams %R 
  *   Identify the overbought and oversold reason.



3.   Bollinger Band
4.   ADX(Average directional movement Index)
  *   ADX is used to find the strength in the perticular tend in the market.



5.   CMF(Chaikin Money Flow)
  *   Determine the buying and selling pressure in the market.






